https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/#:~:text=One%20of%20the%20most%20interesting,multiple%20tasks%20it%20was%20undertrained.

In [1]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [2]:
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'])

Reusing dataset imdb (C:\Users\Emiel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_data = train_data.shuffle().select(range(5000))
test_data = test_data.shuffle().select(range(1000))

In [4]:
tokenizer = RobertaTokenizerFast.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt', num_labels = 3)
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels = 3)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

In [5]:
train_data = train_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
test_data = test_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = 'test',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "steps",
    disable_tqdm = False,
    load_best_model_at_end=False,
    warmup_steps=0,
    weight_decay=0.01,
    fp16 = False,
    dataloader_num_workers = 8,
    run_name = 'roberta-classification')

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Emiel\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 234
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: verrem (use `wandb login --re

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=234, training_loss=0.4084237253564036, metrics={'train_runtime': 427.5323, 'train_samples_per_second': 35.085, 'train_steps_per_second': 0.547, 'total_flos': 3944596358578176.0, 'train_loss': 0.4084237253564036, 'epoch': 3.0})

In [11]:
trainer.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.41812920570373535,
 'eval_accuracy': 0.841,
 'eval_f1': 0.8504233301975541,
 'eval_precision': 0.8233151183970856,
 'eval_recall': 0.8793774319066148,
 'eval_runtime': 13.8058,
 'eval_samples_per_second': 72.433,
 'eval_steps_per_second': 9.054,
 'epoch': 3.0}